# Clinical Trial Adverse Events Data Agent
## Three Example Queries with LLM Testing

This notebook demonstrates the ClinicalTrialDataAgent with three practical queries about adverse events data, plus API key testing.

## Setup: Test API Key First

Make sure you upload your API key to .env

In [52]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if API key is set
api_key = os.getenv('OPENAI_API_KEY')
if api_key:
    print(f"✓ API Key found: {api_key[:20]}...{api_key[-10:]}")
else:
    print("✗ No API key found in .env file")

✓ API Key found: sk-proj-fAJhTgAuzXhC...RGmaACDHkA


In [53]:
# Test the API key with a simple call
from langchain_openai import ChatOpenAI

print("Testing OpenAI API connection...")
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0.2,
        api_key=os.getenv("OPENAI_API_KEY"),
        timeout=10
    )
    
    # Send a quick test message
    response = llm.invoke("Say 'OK' if you can read this")
    print(f"\n✓ API KEY IS WORKING!")
    print(f"Response: {response.content}")
    api_working = True
except Exception as e:
    error_msg = str(e)
    print(f"\n✗ API Connection Failed")
    print(f"Error: {error_msg}")
    print("\nNote: The agent will use fallback pattern matching instead")
    api_working = False

Testing OpenAI API connection...

✓ API KEY IS WORKING!
Response: OK


## Load Data and Initialize Agent

In [54]:
import pandas as pd
from clin_assistant import ClinicalTrialDataAgent

# Load the adverse events data
AE_DATA = pd.read_csv('ae.csv')
print(f"✓ Loaded adverse events dataset")
print(f"  - Records: {AE_DATA.shape[0]}")
print(f"  - Columns: {AE_DATA.shape[1]}")
print(f"\nColumns: {list(AE_DATA.columns)}")

# Initialize the agent
agent = ClinicalTrialDataAgent()
print(f"\n✓ ClinicalTrialDataAgent initialized")

✓ Loaded adverse events dataset
  - Records: 1191
  - Columns: 35

Columns: ['STUDYID', 'DOMAIN', 'USUBJID', 'AESEQ', 'AESPID', 'AETERM', 'AELLT', 'AELLTCD', 'AEDECOD', 'AEPTCD', 'AEHLT', 'AEHLTCD', 'AEHLGT', 'AEHLGTCD', 'AEBODSYS', 'AEBDSYCD', 'AESOC', 'AESOCCD', 'AESEV', 'AESER', 'AEACN', 'AEREL', 'AEOUT', 'AESCAN', 'AESCONG', 'AESDISAB', 'AESDTH', 'AESHOSP', 'AESLIFE', 'AESOD', 'AEDTC', 'AESTDTC', 'AEENDTC', 'AESTDY', 'AEENDY']

✓ ClinicalTrialDataAgent initialized


## Example 1: Find subjects with Moderate Severity Adverse Events

In [55]:
query_1 = "Give me the subjects who had Adverse events of Moderate severity"
print(f"Query: {query_1}\n")

result_1 = agent.query(query_1)

if result_1['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_1['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_1['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_1['filter_applied']}")
    print(f"  Matching Records: {result_1['matching_records']}")
    print(f"  Unique Subjects: {result_1['unique_subjects']}")
else:
    print(f"✗ Error: {result_1.get('error')}")

Query: Give me the subjects who had Adverse events of Moderate severity

✓ Query executed successfully

Parsed Intent:
  Target Column: AESEV
  Filter Value: MODERATE

Results:
  Filter Applied: AESEV = MODERATE
  Matching Records: 378
  Unique Subjects: 136


In [56]:
# Display the filtered dataframe
from IPython.display import display

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_1['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_1['parsed_intent']['filter_value'].upper())
]

# Create a nice formatted display
print(f"📊 Moderate Severity Adverse Events")
print(f"{'='*100}")
print(f"Filter: {result_1['filter_applied']}")
print(f"Total Records: {len(filtered_df)} | Unique Subjects: {len(filtered_df['USUBJID'].unique())}")
print(f"{'='*100}\n")

# Display with pandas styling
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(filtered_df.head())

📊 Moderate Severity Adverse Events
Filter: AESEV = MODERATE
Total Records: 378 | Unique Subjects: 136



,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,AEHLT,AEHLTCD,AEHLGT,AEHLGTCD,AEBODSYS,AEBDSYCD,AESOC,AESOCCD,AESEV,AESER,AEACN,AEREL,AEOUT,AESCAN,AESCONG,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY
5,CDISCPILOT01,AE,01-701-1023,2,E09,ERYTHEMA,LOCALIZED ERYTHEMA,NaN,ERYTHEMA,NaN,HLT_0284,NaN,HLGT_0192,NaN,SKIN AND SUBCUTANEOUS TISSUE DISORDERS,NaN,SKIN AND SUBCUTANEOUS TISSUE DISORDERS,NaN,MODERATE,N,NaN,PROBABLE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2012-08-27,2012-08-07,NaN,3.0,NaN
12,CDISCPILOT01,AE,01-701-1047,1,E06,HIATUS HERNIA,HERNIA HIATAL,NaN,HIATUS HERNIA,NaN,HLT_0159,NaN,HLGT_0109,NaN,GASTROINTESTINAL DISORDERS,NaN,GASTROINTESTINAL DISORDERS,NaN,MODERATE,N,NaN,NONE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2013-02-12,2013-02-12,2013-02-12,1.0,1.0
13,CDISCPILOT01,AE,01-701-1047,2,E06,HIATUS HERNIA,HERNIA HIATAL,NaN,HIATUS HERNIA,NaN,HLT_0159,NaN,HLGT_0109,NaN,GASTROINTESTINAL DISORDERS,NaN,GASTROINTESTINAL DISORDERS,NaN,MODERATE,N,NaN,NONE,RECOVERED/RESOLVED,N,N,N,N,N,N,N,2013-02-25,2013-02-12,2013-02-12,1.0,1.0
16,CDISCPILOT01,AE,01-701-1097,10,E06,APPLICATION SITE PRURITUS,APPLICATION SITE ITCHING,NaN,APPLICATION SITE PRURITUS,NaN,HLT_0317,NaN,HLGT_0338,NaN,GENERAL DISORDERS AND ADMINISTRATION SITE COND...,NaN,GENERAL DISORDERS AND ADMINISTRATION SITE COND...,NaN,MODERATE,N,NaN,POSSIBLE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2014-05-21,2014-02-21,NaN,52.0,NaN
21,CDISCPILOT01,AE,01-701-1097,2,E05,PRURITUS GENERALISED,GENERALIZED PRURITUS,NaN,PRURITUS GENERALISED,NaN,HLT_0123,NaN,HLGT_0689,NaN,SKIN AND SUBCUTANEOUS TISSUE DISORDERS,NaN,SKIN AND SUBCUTANEOUS TISSUE DISORDERS,NaN,MODERATE,N,NaN,POSSIBLE,RECOVERED/RESOLVED,N,N,N,N,N,N,N,2014-02-26,2014-02-20,2014-02-20,51.0,51.0


## Example 2: Find subjects with Cardiac Events

In [57]:
query_2 = "Which subjects experienced cardiac events?"
print(f"Query: {query_2}\n")

result_2 = agent.query(query_2)

if result_2['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_2['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_2['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_2['filter_applied']}")
    print(f"  Matching Records: {result_2['matching_records']}")
    print(f"  Unique Subjects: {result_2['unique_subjects']}")
else:
    print(f"✗ Error: {result_2.get('error')}")

Query: Which subjects experienced cardiac events?

✓ Query executed successfully

Parsed Intent:
  Target Column: AESOC
  Filter Value: CARDIAC DISORDERS

Results:
  Filter Applied: AESOC = CARDIAC DISORDERS
  Matching Records: 91
  Unique Subjects: 44


In [58]:
# Display the filtered dataframe
from IPython.display import display

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_2['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_2['parsed_intent']['filter_value'].upper())
]

# Create a nice formatted display
print(f"📊 Cardiac Adverse Events")
print(f"{'='*100}")
print(f"Filter: {result_2['filter_applied']}")
print(f"Total Records: {len(filtered_df)} | Unique Subjects: {len(filtered_df['USUBJID'].unique())}")
print(f"{'='*100}\n")

# Display with pandas styling
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(filtered_df.head())

📊 Cardiac Adverse Events
Filter: AESOC = CARDIAC DISORDERS
Total Records: 91 | Unique Subjects: 44



,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,AEHLT,AEHLTCD,AEHLGT,AEHLGTCD,AEBODSYS,AEBDSYCD,AESOC,AESOCCD,AESEV,AESER,AEACN,AEREL,AEOUT,AESCAN,AESCONG,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY
3,CDISCPILOT01,AE,01-701-1023,3,E10,ATRIOVENTRICULAR BLOCK SECOND DEGREE,AV BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,HLT_0415,NaN,HLGT_0086,NaN,CARDIAC DISORDERS,NaN,CARDIAC DISORDERS,NaN,MILD,N,NaN,POSSIBLE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2012-08-27,2012-08-26,NaN,22.0,NaN
11,CDISCPILOT01,AE,01-701-1047,4,E09,BUNDLE BRANCH BLOCK LEFT,LEFT BUNDLE BRANCH BLOCK,NaN,BUNDLE BRANCH BLOCK LEFT,NaN,HLT_0281,NaN,HLGT_0721,NaN,CARDIAC DISORDERS,NaN,CARDIAC DISORDERS,NaN,MILD,N,NaN,NONE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2013-03-10,2013-03-10,NaN,27.0,NaN
276,CDISCPILOT01,AE,01-703-1175,1,E01,ATRIOVENTRICULAR BLOCK SECOND DEGREE,HEART BLOCK SECOND DEGREE,NaN,ATRIOVENTRICULAR BLOCK SECOND DEGREE,NaN,HLT_0415,NaN,HLGT_0086,NaN,CARDIAC DISORDERS,NaN,CARDIAC DISORDERS,NaN,SEVERE,N,NaN,NONE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2013-12-18,2013-12-18,NaN,-2.0,NaN
295,CDISCPILOT01,AE,01-703-1299,8,E07,MYOCARDIAL INFARCTION,MYOCARDIAL INFARCTION,NaN,MYOCARDIAL INFARCTION,NaN,HLT_0649,NaN,HLGT_0688,NaN,CARDIAC DISORDERS,NaN,CARDIAC DISORDERS,NaN,MODERATE,N,NaN,REMOTE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2013-03-13,2013-03-13,NaN,183.0,NaN
296,CDISCPILOT01,AE,01-703-1299,5,E06,TACHYCARDIA,TACHYCARDIA,NaN,TACHYCARDIA,NaN,HLT_0082,NaN,HLGT_0574,NaN,CARDIAC DISORDERS,NaN,CARDIAC DISORDERS,NaN,MILD,N,NaN,REMOTE,NOT RECOVERED/NOT RESOLVED,N,N,N,N,N,N,N,2013-03-01,2013-03-01,NaN,171.0,NaN


## Example 3 (COMPLEX): Find me subjects that had a serious AE

In [59]:
query_3 = "Find me subjects that had a serious AE"
print(f"Query: {query_3}\n")

result_3 = agent.query(query_3)

if result_3['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  Target Column: {result_3['parsed_intent']['target_column']}")
    print(f"  Filter Value: {result_3['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  Filter Applied: {result_3['filter_applied']}")
    print(f"  Matching Records: {result_3['matching_records']}")
    print(f"  Unique Subjects: {result_3['unique_subjects']}")
else:
    print(f"✗ Error: {result_3.get('error')}")

Query: Find me subjects that had a serious AE

✓ Query executed successfully

Parsed Intent:
  Target Column: AESER
  Filter Value: Y

Results:
  Filter Applied: AESER = Y
  Matching Records: 3
  Unique Subjects: 3


In [60]:
# Display the filtered dataframe
from IPython.display import display

# Filter AE_DATA based on the query results
filtered_df = AE_DATA[
    (AE_DATA[result_3['parsed_intent']['target_column']].astype(str).str.upper() == 
     result_3['parsed_intent']['filter_value'].upper())
]

# Create a nice formatted display
print(f"📊 Recovered/Resolved Adverse Events")
print(f"{'='*100}")
print(f"Filter: {result_3['filter_applied']}")
print(f"Total Records: {len(filtered_df)} | Unique Subjects: {len(filtered_df['USUBJID'].unique())}")
print(f"{'='*100}\n")

# Display with pandas styling
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(filtered_df)

📊 Recovered/Resolved Adverse Events
Filter: AESER = Y
Total Records: 3 | Unique Subjects: 3



,STUDYID,DOMAIN,USUBJID,AESEQ,AESPID,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,AEHLT,AEHLTCD,AEHLGT,AEHLGTCD,AEBODSYS,AEBDSYCD,AESOC,AESOCCD,AESEV,AESER,AEACN,AEREL,AEOUT,AESCAN,AESCONG,AESDISAB,AESDTH,AESHOSP,AESLIFE,AESOD,AEDTC,AESTDTC,AEENDTC,AESTDY,AEENDY
688,CDISCPILOT01,AE,01-709-1424,1,E05,SYNCOPE,SYNCOPE,NaN,SYNCOPE,NaN,HLT_0440,NaN,HLGT_0310,NaN,NERVOUS SYSTEM DISORDERS,NaN,NERVOUS SYSTEM DISORDERS,NaN,MODERATE,Y,NaN,POSSIBLE,RECOVERED/RESOLVED,N,N,N,N,N,Y,N,2013-03-08,2013-03-07,2013-03-07,5.0,5.0
1131,CDISCPILOT01,AE,01-718-1170,5,E03,SYNCOPE,SYNCOPE,NaN,SYNCOPE,NaN,HLT_0440,NaN,HLGT_0310,NaN,NERVOUS SYSTEM DISORDERS,NaN,NERVOUS SYSTEM DISORDERS,NaN,SEVERE,Y,NaN,PROBABLE,RECOVERED/RESOLVED,N,N,N,N,Y,N,N,2013-11-03,2013-10-12,2013-10-13,27.0,28.0
1174,CDISCPILOT01,AE,01-718-1371,4,E11,PARTIAL SEIZURES WITH SECONDARY GENERALISATION,PARTIAL SEIZURES WITH SECONDARY GENERALIZATION,NaN,PARTIAL SEIZURES WITH SECONDARY GENERALISATION,NaN,HLT_0603,NaN,HLGT_0719,NaN,NERVOUS SYSTEM DISORDERS,NaN,NERVOUS SYSTEM DISORDERS,NaN,SEVERE,Y,NaN,NONE,RECOVERED/RESOLVED,N,N,N,N,Y,N,N,2013-06-20,2013-06-02,2013-06-05,38.0,41.0


## Summary Comparison

In [61]:
# Create summary table
from IPython.display import display

summary_data = [
    {
        'Query': 'Moderate Severity',
        'Filter': result_1['filter_applied'],
        'Records': result_1['matching_records'],
        'Unique Subjects': result_1['unique_subjects']
    },
    {
        'Query': 'Cardiac Events',
        'Filter': result_2['filter_applied'],
        'Records': result_2['matching_records'],
        'Unique Subjects': result_2['unique_subjects']
    },
    {
        'Query': 'Recovered Events',
        'Filter': result_3['filter_applied'],
        'Records': result_3['matching_records'],
        'Unique Subjects': result_3['unique_subjects']
    }
]

summary_df = pd.DataFrame(summary_data)
print(f"📋 Summary of All Queries")
print(f"{'='*140}\n")
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    display(summary_df)

📋 Summary of All Queries



,Query,Filter,Records,Unique Subjects
0,Moderate Severity,AESEV = MODERATE,378,136
1,Cardiac Events,AESOC = CARDIAC DISORDERS,91,44
2,Recovered Events,AESER = Y,3,3


## Notes

- The agent can use either LLM-based parsing or fallback pattern matching
- Each query uses minimal tokens since it just converts natural language to structured filters
- The agent returns unique subject counts and complete subject IDs for further analysis

# Clinical Trial Adverse Events Data Agent
## Three Example Queries

This notebook demonstrates the ClinicalTrialDataAgent with three practical queries about adverse events data.

### Setup: Import and Initialize

In [62]:
import os
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Load the adverse events data
AE_DATA = pd.read_csv('ae.csv')
print(f"Loaded adverse events dataset: {AE_DATA.shape[0]} records, {AE_DATA.shape[1]} columns")
print(f"\nFirst few rows:")
print(AE_DATA.head())
print(f"\nColumns: {list(AE_DATA.columns)}")

Loaded adverse events dataset: 1191 records, 35 columns

First few rows:
        STUDYID DOMAIN      USUBJID  AESEQ AESPID  \
0  CDISCPILOT01     AE  01-701-1015      1    E07   
1  CDISCPILOT01     AE  01-701-1015      2    E08   
2  CDISCPILOT01     AE  01-701-1015      3    E06   
3  CDISCPILOT01     AE  01-701-1023      3    E10   
4  CDISCPILOT01     AE  01-701-1023      1    E08   

                                 AETERM                     AELLT  AELLTCD  \
0             APPLICATION SITE ERYTHEMA  APPLICATION SITE REDNESS      NaN   
1             APPLICATION SITE PRURITUS  APPLICATION SITE ITCHING      NaN   
2                             DIARRHOEA                  DIARRHEA      NaN   
3  ATRIOVENTRICULAR BLOCK SECOND DEGREE    AV BLOCK SECOND DEGREE      NaN   
4                              ERYTHEMA                  ERYTHEMA      NaN   

                                AEDECOD  AEPTCD  ... AESDISAB  AESDTH AESHOSP  \
0             APPLICATION SITE ERYTHEMA     NaN  ...      

In [63]:
# Import the ClinicalTrialDataAgent
from clin_assistant import ClinicalTrialDataAgent

# Initialize the agent
agent = ClinicalTrialDataAgent()
print("✓ ClinicalTrialDataAgent initialized")

✓ ClinicalTrialDataAgent initialized


## Example 1: Find subjects with Moderate Severity Adverse Events

**Query:** "Give me the subjects who had Adverse events of Moderate severity"

In [64]:
query_1 = "Give me the subjects who had Adverse events of Moderate severity"
print(f"Query: {query_1}\n")

result_1 = agent.query(query_1)

if result_1['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_1['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_1['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_1['filter_applied']}")
    print(f"  - Matching Records: {result_1['matching_records']}")
    print(f"  - Unique Subjects: {result_1['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_1['subject_ids'])}) :")
    for subj_id in result_1['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_1['subject_ids']) > 15:
        print(f"  ... and {len(result_1['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_1.get('error')}")

Query: Give me the subjects who had Adverse events of Moderate severity

✓ Query executed successfully

Parsed Intent:
  - Target Column: AESEV
  - Filter Value: MODERATE

Results:
  - Filter Applied: AESEV = MODERATE
  - Matching Records: 378
  - Unique Subjects: 136

Subject IDs (first 15 of 136) :
  - 01-701-1023
  - 01-701-1047
  - 01-701-1097
  - 01-701-1111
  - 01-701-1115
  - 01-701-1133
  - 01-701-1146
  - 01-701-1148
  - 01-701-1180
  - 01-701-1181
  - 01-701-1188
  - 01-701-1192
  - 01-701-1211
  - 01-701-1239
  - 01-701-1294
  ... and 121 more


In [65]:
# Show sample data from results
print("Sample Adverse Events (first 5):")
print()
for i, record in enumerate(result_1['sample_data'], 1):
    print(f"{i}. {record}")
    print()

Sample Adverse Events (first 5):

1. {'USUBJID': '01-701-1023', 'AETERM': 'ERYTHEMA', 'AESEV': 'MODERATE', 'AESOC': 'SKIN AND SUBCUTANEOUS TISSUE DISORDERS'}

2. {'USUBJID': '01-701-1047', 'AETERM': 'HIATUS HERNIA', 'AESEV': 'MODERATE', 'AESOC': 'GASTROINTESTINAL DISORDERS'}

3. {'USUBJID': '01-701-1047', 'AETERM': 'HIATUS HERNIA', 'AESEV': 'MODERATE', 'AESOC': 'GASTROINTESTINAL DISORDERS'}

4. {'USUBJID': '01-701-1097', 'AETERM': 'APPLICATION SITE PRURITUS', 'AESEV': 'MODERATE', 'AESOC': 'GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS'}

5. {'USUBJID': '01-701-1097', 'AETERM': 'PRURITUS GENERALISED', 'AESEV': 'MODERATE', 'AESOC': 'SKIN AND SUBCUTANEOUS TISSUE DISORDERS'}



## Example 2: Find subjects with Cardiac Events

**Query:** "Which subjects experienced cardiac events?"

In [66]:
query_2 = "Which subjects experienced cardiac events?"
print(f"Query: {query_2}\n")

result_2 = agent.query(query_2)

if result_2['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_2['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_2['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_2['filter_applied']}")
    print(f"  - Matching Records: {result_2['matching_records']}")
    print(f"  - Unique Subjects: {result_2['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_2['subject_ids'])}) :")
    for subj_id in result_2['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_2['subject_ids']) > 15:
        print(f"  ... and {len(result_2['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_2.get('error')}")

Query: Which subjects experienced cardiac events?

✓ Query executed successfully

Parsed Intent:
  - Target Column: AESOC
  - Filter Value: CARDIAC DISORDERS

Results:
  - Filter Applied: AESOC = CARDIAC DISORDERS
  - Matching Records: 91
  - Unique Subjects: 44

Subject IDs (first 15 of 44) :
  - 01-701-1023
  - 01-701-1047
  - 01-703-1175
  - 01-703-1299
  - 01-703-1335
  - 01-704-1008
  - 01-704-1017
  - 01-704-1025
  - 01-704-1065
  - 01-704-1266
  - 01-705-1349
  - 01-707-1206
  - 01-708-1084
  - 01-708-1178
  - 01-708-1213
  ... and 29 more


In [67]:
# Show sample data from results
print("Sample Cardiac Adverse Events (first 5):")
print()
for i, record in enumerate(result_2['sample_data'], 1):
    print(f"{i}. {record}")
    print()

Sample Cardiac Adverse Events (first 5):

1. {'USUBJID': '01-701-1023', 'AETERM': 'ATRIOVENTRICULAR BLOCK SECOND DEGREE', 'AESEV': 'MILD', 'AESOC': 'CARDIAC DISORDERS'}

2. {'USUBJID': '01-701-1047', 'AETERM': 'BUNDLE BRANCH BLOCK LEFT', 'AESEV': 'MILD', 'AESOC': 'CARDIAC DISORDERS'}

3. {'USUBJID': '01-703-1175', 'AETERM': 'ATRIOVENTRICULAR BLOCK SECOND DEGREE', 'AESEV': 'SEVERE', 'AESOC': 'CARDIAC DISORDERS'}

4. {'USUBJID': '01-703-1299', 'AETERM': 'MYOCARDIAL INFARCTION', 'AESEV': 'MODERATE', 'AESOC': 'CARDIAC DISORDERS'}

5. {'USUBJID': '01-703-1299', 'AETERM': 'TACHYCARDIA', 'AESEV': 'MILD', 'AESOC': 'CARDIAC DISORDERS'}



## Example 3: Find subjects with Recovered Adverse Events

**Query:** "Show me all subjects with recovered adverse events"

In [68]:
query_3 = "Show me all subjects with recovered adverse events"
print(f"Query: {query_3}\n")

result_3 = agent.query(query_3)

if result_3['success']:
    print("✓ Query executed successfully\n")
    print(f"Parsed Intent:")
    print(f"  - Target Column: {result_3['parsed_intent']['target_column']}")
    print(f"  - Filter Value: {result_3['parsed_intent']['filter_value']}")
    
    print(f"\nResults:")
    print(f"  - Filter Applied: {result_3['filter_applied']}")
    print(f"  - Matching Records: {result_3['matching_records']}")
    print(f"  - Unique Subjects: {result_3['unique_subjects']}")
    
    print(f"\nSubject IDs (first 15 of {len(result_3['subject_ids'])}) :")
    for subj_id in result_3['subject_ids'][:15]:
        print(f"  - {subj_id}")
    if len(result_3['subject_ids']) > 15:
        print(f"  ... and {len(result_3['subject_ids']) - 15} more")
else:
    print(f"✗ Error: {result_3.get('error')}")

Query: Show me all subjects with recovered adverse events

✓ Query executed successfully

Parsed Intent:
  - Target Column: AEOUT
  - Filter Value: RECOVERED/RESOLVED

Results:
  - Filter Applied: AEOUT = RECOVERED/RESOLVED
  - Matching Records: 465
  - Unique Subjects: 161

Subject IDs (first 15 of 161) :
  - 01-701-1015
  - 01-701-1023
  - 01-701-1047
  - 01-701-1097
  - 01-701-1111
  - 01-701-1115
  - 01-701-1130
  - 01-701-1146
  - 01-701-1148
  - 01-701-1153
  - 01-701-1180
  - 01-701-1181
  - 01-701-1192
  - 01-701-1203
  - 01-701-1211
  ... and 146 more


In [69]:
# Show sample data from results
print("Sample Recovered Adverse Events (first 5):")
print()
for i, record in enumerate(result_3['sample_data'], 1):
    print(f"{i}. {record}")
    print()

Sample Recovered Adverse Events (first 5):

1. {'USUBJID': '01-701-1015', 'AETERM': 'DIARRHOEA', 'AESEV': 'MILD', 'AESOC': 'GASTROINTESTINAL DISORDERS'}

2. {'USUBJID': '01-701-1023', 'AETERM': 'ERYTHEMA', 'AESEV': 'MILD', 'AESOC': 'SKIN AND SUBCUTANEOUS TISSUE DISORDERS'}

3. {'USUBJID': '01-701-1047', 'AETERM': 'HIATUS HERNIA', 'AESEV': 'MODERATE', 'AESOC': 'GASTROINTESTINAL DISORDERS'}

4. {'USUBJID': '01-701-1097', 'AETERM': 'APPLICATION SITE VESICLES', 'AESEV': 'MILD', 'AESOC': 'GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS'}

5. {'USUBJID': '01-701-1097', 'AETERM': 'NASAL CONGESTION', 'AESEV': 'MILD', 'AESOC': 'RESPIRATORY, THORACIC AND MEDIASTINAL DISORDERS'}

